In [1]:
import selenium.webdriver
import bs4 as bs
import time

def load_page(url="https://www.reddit.com/r/prolife"):
    driver.get(url)

    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = bs.BeautifulSoup(driver.page_source,"html.parser")

    links = []
    for link in soup.find_all("a",{"data-click-id":"body"}):
        if link.has_attr('href'):
            if "/prolife/comments" in link["href"]:
                links.append(link['href'])
    return links


def get_data(text,f):
    html = bs.BeautifulSoup(text)
    comments =  html.find_all("div",{'class': re.compile(".*s136il31-0.*")})

    for com in comments:
        #print(com)
        level = len(com.find_all("div",{"class": re.compile(".*efNcNS.*")}))
        com = com.find("div",{'class': re.compile('.*bbMwOF')})

        user = com.find("a",{"href": re.compile(".*user.*")}).text
        comment = " ".join([_.text for _ in com.find_all("p",{"class": re.compile(".*iEJDri.*")})])
        points = com.find("span",{"class": re.compile(".*cFQOcm.*")}).text[:-7]
        

        f.write("{}\t{}\t{}\t{}\n".format(user,level,points,comment.replace("\t"," ").replace("\n"," ").replace("\r"," ")))
        
        

def main(reddit_group):
    driver = selenium.webdriver.Chrome(executable_path="./chromedriver")
    links = load_page(reddit_group)
    
    time.sleep(10)

    with open("./results.csv","w+") as f:
        for link in links:
            url = "https://www.reddit.com" + link
            driver.get(url)
            get_data(driver.page_source,f)

In [ ]:
main("https://www.reddit.com/r/prolife")